<a href="https://colab.research.google.com/github/dimna21/ML_Assignment4/blob/main/FER2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dimna21 (dimna21-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Load the CSV
csv_path = "/content/drive/MyDrive/FER_data/fer2013/fer2013.csv"
df = pd.read_csv(csv_path)

# Split by Usage
df_train = df[df['Usage']=="Training"].copy()
df_val   = df[df['Usage']=="PublicTest"].copy()
df_test  = df[df['Usage']=="PrivateTest"].copy()

In [ ]:
# Balance function: upsampling & random ±10 intensity shifts
def balance_dataset(df, target_count, img_shape=(48,48)):
    def augment(pix_str):
        arr = np.fromstring(pix_str, sep=' ', dtype=int).reshape(img_shape)
        shift = np.random.randint(-10, 11)
        arr = np.clip(arr + shift, 0, 255).astype(int)
        return ' '.join(map(str, arr.ravel()))
    parts = [df]
    for emo, grp in df.groupby('emotion'):
        n = len(grp)
        if n < target_count:
            extra = grp.sample(n=target_count-n, replace=True).copy()
            extra['pixels'] = extra['pixels'].map(augment)
            parts.append(extra)
    return pd.concat(parts, ignore_index=True)


In [ ]:
# Dataset class
class FERDataset(Dataset):
    def __init__(self, dataframe):
        self.pixels = dataframe['pixels'].values
        self.labels = dataframe['emotion'].values.astype(int)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        arr = np.fromstring(self.pixels[idx], sep=' ', dtype=np.uint8).reshape(48,48)
        arr = arr.astype(np.float32) / 255.0
        tensor = torch.from_numpy(arr).unsqueeze(0)  # shape [1,48,48]
        return tensor, self.labels[idx]


In [ ]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
# DataLoaders
max_count = df_train['emotion'].value_counts().max()
balanced_train = balance_dataset(df_train, target_count=max_count)

#train_ds = FERDataset(df_train)
train_ds = FERDataset(balanced_train)
val_ds = FERDataset(df_val)
test_ds = FERDataset(df_test)

batch_size = 256
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
val_dl = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)
test_dl = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=2)

# Class names
class_names = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

In [ ]:
balanced_train['emotion'].value_counts()

,count
emotion,
0,7215
2,7215
4,7215
6,7215
3,7215
5,7215
1,7215


In [ ]:
import torch
from tqdm import tqdm
import wandb
from sklearn.metrics import confusion_matrix, f1_score
import matplotlib.pyplot as plt
import numpy as np

def train_model(model, train_loader, val_loader,
                criterion, optimizer, device,
                epochs=5, class_names=None):

    wandb.init(
        project="ML_Assignment4",
        config={
            "epochs": epochs,
            "batch_size": train_loader.batch_size,
            "optimizer": optimizer.__class__.__name__,
            "lr": optimizer.param_groups[0]["lr"],
            "criterion": criterion.__class__.__name__,
        },
    )
    cfg = wandb.config
    wandb.watch(model, log="all", log_freq=100)
    model.to(device)

    train_loss_plot, val_loss_plot = [], []
    train_acc_plot,  val_acc_plot  = [], []

    for epoch in range(1, cfg.epochs + 1):
        # — TRAIN —
        model.train()
        running_loss = running_correct = running_total = 0
        for X, y in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            running_loss    += loss.item() * X.size(0)
            preds            = out.argmax(dim=1)
            running_correct += preds.eq(y).sum().item()
            running_total   += y.size(0)

        train_loss = running_loss / running_total
        train_acc  = running_correct / running_total

        # — VALIDATE —
        model.eval()
        val_running_loss = val_running_correct = val_running_total = 0
        all_preds, all_targets = [], []
        with torch.no_grad():
            for X, y in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
                X, y = X.to(device), y.to(device)
                out = model(X)
                loss = criterion(out, y)

                val_running_loss    += loss.item() * X.size(0)
                preds                = out.argmax(dim=1)
                val_running_correct += preds.eq(y).sum().item()
                val_running_total   += y.size(0)

                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(y.cpu().numpy())

        val_loss = val_running_loss / val_running_total
        val_acc  = val_running_correct / val_running_total

        # — SCALARS & F1 —
        cm = confusion_matrix(all_targets, all_preds)
        f1_per_class = f1_score(all_targets, all_preds, average=None)

        log_data = {
            "train_loss": train_loss,
            "train_acc":  train_acc,
            "val_loss":   val_loss,
            "val_acc":    val_acc,
        }
        for i, name in enumerate(class_names):
            log_data[f"f1_{name}"] = f1_per_class[i]

        wandb.log(log_data, step=epoch)

        # — 7×7 Confusion Matrix Plot & Log —
        fig_cm, ax = plt.subplots(figsize=(6,6))
        im = ax.imshow(cm, interpolation='nearest', cmap='Blues')
        fig_cm.colorbar(im, ax=ax)

        ax.set_xticks(np.arange(len(class_names)))
        ax.set_yticks(np.arange(len(class_names)))
        ax.set_xticklabels(class_names, rotation=45, ha='right')
        ax.set_yticklabels(class_names)

        for i in range(len(class_names)):
            for j in range(len(class_names)):
                ax.text(j, i, cm[i, j],
                        ha='center', va='center')

        ax.set_xlabel('Predicted')
        ax.set_ylabel('Actual')
        ax.set_title(f'Epoch {epoch} Confusion Matrix')

        wandb.log({"confusion_matrix": wandb.Image(fig_cm)}, step=epoch)
        plt.close(fig_cm)

        # — PRINT & STORE FOR CURVES —
        print(
            f"Epoch {epoch}/{cfg.epochs} — "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}  |  "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
        )
        print("-" * 60)

        train_loss_plot.append(train_loss)
        val_loss_plot.append(val_loss)
        train_acc_plot.append(train_acc)
        val_acc_plot.append(val_acc)

    # — PLOT & LOG LOSS/ACC CURVES TO W&B —
    epochs_range = list(range(1, cfg.epochs + 1))

    # Loss curve
    fig1, ax1 = plt.subplots()
    ax1.plot(epochs_range, train_loss_plot, label="Train Loss")
    ax1.plot(epochs_range, val_loss_plot,   label="Val Loss")
    ax1.set(title="Loss vs Epoch", xlabel="Epoch", ylabel="Loss")
    ax1.legend()
    wandb.log({"loss_curve": wandb.Image(fig1)})
    plt.close(fig1)

    # Accuracy curve
    fig2, ax2 = plt.subplots()
    ax2.plot(epochs_range, train_acc_plot, label="Train Acc")
    ax2.plot(epochs_range, val_acc_plot,   label="Val Acc")
    ax2.set(title="Accuracy vs Epoch", xlabel="Epoch", ylabel="Accuracy")
    ax2.legend()
    wandb.log({"acc_curve": wandb.Image(fig2)})
    plt.close(fig2)

    return model


In [ ]:
class BaselineModel(nn.Module):
    def __init__(self):
      super().__init__()

      self.conv1 = nn.Conv2d(1, 32, kernel_size = 3, padding = 1)
      self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
      self.conv3 = nn.Conv2d(64, 128, kernel_size = 3, padding = 1)
      self.pooling = nn.MaxPool2d(2,2)
      self.relu = nn.ReLU()

      self.flatten = nn.Flatten()
      self.linear = nn.Linear((128 * 6 * 6), 128)
      self.output = nn.Linear(128, 7)


    def forward(self, x):
      x = self.conv1(x)
      x = self.pooling(x)
      x = self.relu(x)
      x = self.conv2(x)
      x = self.pooling(x)
      x = self.relu(x)
      x = self.conv3(x)
      x = self.pooling(x)
      x = self.relu(x)
      x = self.flatten(x)
      x = self.linear(x)
      x = self.output(x)

      return x

#Run with dataset balancing:
#https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/7amufnzs?nw=nwuserdimna21

#Run without dataset balancing:
#https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/dnyp6qje
#20 epochs instead of 10:
#https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/0axoxyj9?nw=nwuserdimna21

In [ ]:
baseline_model = BaselineModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(baseline_model.parameters(), lr=1e-3)
trained = train_model(
    baseline_model,
    train_dl,
    val_dl,
    criterion,
    optimizer,
    device,
    epochs=20,
    class_names=class_names
)

f1_Angry,▁▂▅▆▆▆▇▇█▇
f1_Disgust,▁▁▂▄▅▇▅▇██
f1_Fear,▁▃▄▅▅▆▇█▆█
f1_Happy,▁▃▄▇▆▇█▇██
f1_Neutral,▁▂▄▆▇██▇▇█
f1_Sad,▁▅▃▇█▅█▆▇▆
f1_Surprise,▁▃▅▇▇▇▇█▇█
train_acc,▁▃▄▅▆▆▇▇██
train_loss,█▆▅▄▃▃▂▂▁▁
val_acc,▁▄▄▇▇▇████
val_loss,█▅▄▂▁▁▁▂▂▂


Epoch 1 [Val]: 100%|██████████| 29/29 [00:01<00:00, 25.32it/s]


Epoch 1/20 — Train Loss: 1.6812, Train Acc: 0.3272  |  Val Loss: 1.5147, Val Acc: 0.4235
------------------------------------------------------------


Epoch 2 [Val]: 100%|██████████| 29/29 [00:00<00:00, 46.90it/s]


Epoch 2/20 — Train Loss: 1.4493, Train Acc: 0.4449  |  Val Loss: 1.3864, Val Acc: 0.4762
------------------------------------------------------------


...

Epoch 18/20 — Train Loss: 0.4636, Train Acc: 0.8318  |  Val Loss: 1.8411, Val Acc: 0.5414
------------------------------------------------------------


Epoch 19 [Val]: 100%|██████████| 29/29 [00:00<00:00, 50.49it/s]


Epoch 19/20 — Train Loss: 0.4270, Train Acc: 0.8452  |  Val Loss: 1.8972, Val Acc: 0.5511
------------------------------------------------------------


Epoch 20 [Val]: 100%|██████████| 29/29 [00:00<00:00, 48.18it/s]


Epoch 20/20 — Train Loss: 0.3815, Train Acc: 0.8583  |  Val Loss: 2.1096, Val Acc: 0.5411
------------------------------------------------------------


In [ ]:
class ImprovedModel(nn.Module):
    def __init__(self, dropout_p=0.5):
        super().__init__()
        # — conv backbone with 4 pooling stages —
        self.conv1    = nn.Conv2d(1,   32, 3, padding=1, bias=False)
        self.bn1      = nn.BatchNorm2d(32)
        self.conv2    = nn.Conv2d(32,  64, 3, padding=1, bias=False)
        self.bn2      = nn.BatchNorm2d(64)
        self.conv3    = nn.Conv2d(64, 128, 3, padding=1, bias=False)
        self.bn3      = nn.BatchNorm2d(128)
        self.conv4    = nn.Conv2d(128,256, 3, padding=1, bias=False)
        self.bn4      = nn.BatchNorm2d(256)

        self.pool     = nn.MaxPool2d(2,2)
        self.relu     = nn.ReLU(inplace=True)
        self.drop_conv= nn.Dropout2d(dropout_p)

        # — two extra conv layers (no further pooling) —
        self.conv5    = nn.Conv2d(256, 512, 3, padding=1, bias=False)
        self.bn5      = nn.BatchNorm2d(512)
        self.conv6    = nn.Conv2d(512, 512, 3, padding=1, bias=False)
        self.bn6      = nn.BatchNorm2d(512)

        # — richer head with 4 FCs —
        self.flatten  = nn.Flatten()
        self.fc1      = nn.Linear(512 * 3 * 3, 512, bias=False)
        self.bn_fc1   = nn.BatchNorm1d(512)
        self.drop1    = nn.Dropout(dropout_p)
        self.fc2      = nn.Linear(512, 256, bias=False)
        self.bn_fc2   = nn.BatchNorm1d(256)
        self.drop2    = nn.Dropout(dropout_p)
        self.fc3      = nn.Linear(256, 128, bias=False)
        self.bn_fc3   = nn.BatchNorm1d(128)
        self.drop3    = nn.Dropout(dropout_p)
        self.fc4      = nn.Linear(128, 64, bias=False)
        self.bn_fc4   = nn.BatchNorm1d(64)
        self.drop4    = nn.Dropout(dropout_p)
        self.output   = nn.Linear(64, 7)

    def forward(self, x):
        # conv blocks 1–4 with pooling
        for conv, bn in [(self.conv1,self.bn1),
                         (self.conv2,self.bn2),
                         (self.conv3,self.bn3),
                         (self.conv4,self.bn4)]:
            x = conv(x)
            x = bn(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.drop_conv(x)

        # extra conv blocks 5–6 (no more pooling)
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu(x)
        x = self.drop_conv(x)

        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu(x)
        x = self.drop_conv(x)

        # FC head
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = self.bn_fc2(x)
        x = self.relu(x)
        x = self.drop2(x)

        x = self.fc3(x)
        x = self.bn_fc3(x)
        x = self.relu(x)
        x = self.drop3(x)

        x = self.fc4(x)
        x = self.bn_fc4(x)
        x = self.relu(x)
        x = self.drop4(x)

        x = self.output(x)
        return x


#Run with added batchnorms
#https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/cuxqkkdm?nw=nwuserdimna21
#topped out at 70%/58%

#Run with added batchnorms and dropouts
#10 epochs:  https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/iactnmo6?nw=nwuserdimna21
#20 epochs:  https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/ydlxqyzi?nw=nwuserdimna21
#started overfitting after reaching 62%/60% accuracies on train/validation

#Run with added 4th convolutional block:
#https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/lqfkix8t
#no improvements, got 62%/60%

#Run with a more complex head of 512->256->128
#https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/65qgarhf
#slight improvement, got 64%/61%

#Run with added residual blocks
#20 epochs: https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/op1fbc6g
#40 epochs: https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/cyn9apy4
#no improvement

#Run without residual blocks, with augmented data and increased complexity: 2 extra conv layers and 4 FCs in head
#40 epochs, 0.3 dropout: https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/ofsajdfh?nw=nwuserdimna21
#40 epochs, 0.5 dropout: https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/itxksjhl
#40 epochs, 0.5 dropout, double batch size (256): https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/x225ygh0

In [ ]:
import torch.nn as nn

class ResidualBlock(nn.Module):
    """
    A basic residual block with two 3x3 conv layers and an optional downsample.
    """
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, dropout_p=0.0):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout2d(dropout_p) if dropout_p > 0 else nn.Identity()

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        out = self.dropout(out)
        return out

class GigaModel(nn.Module):
    def __init__(self, dropout_p=0.3):
        super().__init__()
        # define ReLU for use in head
        self.relu = nn.ReLU(inplace=True)
        # initial conv to expand grayscale to 32 channels
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        # residual layers: doubling channels and downsampling spatially at each stage
        self.layer1 = ResidualBlock(32,  32, stride=1, dropout_p=dropout_p)
        self.layer2 = ResidualBlock(32,  64, stride=2, dropout_p=dropout_p)
        self.layer3 = ResidualBlock(64, 128, stride=2, dropout_p=dropout_p)
        self.layer4 = ResidualBlock(128,256, stride=2, dropout_p=dropout_p)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        self.flatten = nn.Flatten()
        self.fc1     = nn.Linear(256 * ResidualBlock.expansion, 512, bias=False)
        self.bn_fc1  = nn.BatchNorm1d(512)
        self.drop1   = nn.Dropout(dropout_p)
        self.fc2     = nn.Linear(512, 256, bias=False)
        self.bn_fc2  = nn.BatchNorm1d(256)
        self.drop2   = nn.Dropout(dropout_p)
        self.fc3     = nn.Linear(256, 128, bias=False)
        self.bn_fc3  = nn.BatchNorm1d(128)
        self.drop3   = nn.Dropout(dropout_p)
        self.output  = nn.Linear(128, 7)

    def forward(self, x):
        # initial conv
        x = self.conv1(x)
        # residual conv backbone with downsamples
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # head
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.bn_fc2(x)
        x = self.relu(x)
        x = self.drop2(x)
        x = self.fc3(x)
        x = self.bn_fc3(x)
        x = self.relu(x)
        x = self.drop3(x)
        x = self.output(x)
        return x

#training pt1: https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/8a4wuiau?nw=nwuserdimna21
#training pt2: https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/127aogq7

In [ ]:
improved_model = ImprovedModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(improved_model.parameters(), lr=1e-3)
trained = train_model(
    improved_model,
    train_dl,
    val_dl,
    criterion,
    optimizer,
    device,
    epochs=40,
    class_names=class_names
)

Epoch 1 [Val]: 100%|██████████| 15/15 [00:01<00:00, 14.57it/s]


Epoch 1/40 — Train Loss: 1.9672, Train Acc: 0.1561  |  Val Loss: 1.9162, Val Acc: 0.1914
------------------------------------------------------------


Epoch 2 [Val]: 100%|██████████| 15/15 [00:00<00:00, 29.18it/s]


Epoch 2/40 — Train Loss: 1.9370, Train Acc: 0.1686  |  Val Loss: 1.9108, Val Acc: 0.1962
------------------------------------------------------------


Epoch 3 [Val]: 100%|██████████| 15/15 [00:00<00:00, 18.22it/s]


...

Epoch 38/40 — Train Loss: 1.1365, Train Acc: 0.5599  |  Val Loss: 1.2715, Val Acc: 0.5235
------------------------------------------------------------


Epoch 39 [Val]: 100%|██████████| 15/15 [00:01<00:00, 14.57it/s]


Epoch 39/40 — Train Loss: 1.1286, Train Acc: 0.5650  |  Val Loss: 1.2506, Val Acc: 0.5249
------------------------------------------------------------


Epoch 40 [Val]: 100%|██████████| 15/15 [00:00<00:00, 28.93it/s]


Epoch 40/40 — Train Loss: 1.1290, Train Acc: 0.5610  |  Val Loss: 1.2449, Val Acc: 0.5286
------------------------------------------------------------


In [ ]:
torch.save(improved_model.state_dict(), "checkpoint40.pth")

In [ ]:
improved_model = GigaModel().to(device)
improved_model.load_state_dict(torch.load("checkpoint40.pth"))


<All keys matched successfully>

In [ ]:
trained = train_model(
    improved_model, train_dl, val_dl,
    criterion, optimizer, device,
    epochs=40,
    class_names=class_names
)

f1_Angry,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▅▂▃▃▃▆▄▃▆▅▆▆▅▆▆▆▇▇▇█▇████
f1_Disgust,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1_Fear,▁▂▂▂▂▂█▄▃█▂▃▆▃▄▄▂▅▄▅▅▅▃▄▄▃▃▅▄▆▆▆▆▆▆▆▇▆▆▆
f1_Happy,▂▂▂▂▃▃▁▄▄▅▆▆▇▆▇▇▆▆▇▇▇▆▇▇▇█▇▇▇▇██████████
f1_Neutral,▁▁▁▂▃▃▃▅▅▅▆▃▆▆▇▇▃▆▇▇▆▆▅▇▇▇▆▇▇▇██████████
f1_Sad,▂▃▁▂▃▂▄▃▅▅▆▇▇▆▆▆▇▆▇▅▇▇▇▇▇▇▇███▇▇████████
f1_Surprise,▃▁▁▄▆▇▄▇▅▇▇▇▇█▇█▇███▇█▇█████▇███████████
train_acc,▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████
train_loss,█▇▇▇▇▆▆▆▅▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_acc,▂▂▂▂▃▃▁▄▄▅▅▅▆▆▆▆▅▆▆▆▇▇▆▇▇▇▇▇▇███████████
val_loss,▆▇▇▆▅▅█▄▅▄▃▃▃▃▃▃▃▂▃▃▂▂▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁


Epoch 1 [Val]: 100%|██████████| 29/29 [00:03<00:00,  7.39it/s]


Epoch 1/40 — Train Loss: 1.2203, Train Acc: 0.5256  |  Val Loss: 1.2049, Val Acc: 0.5313
------------------------------------------------------------


Epoch 2 [Val]: 100%|██████████| 29/29 [00:03<00:00,  7.87it/s]


Epoch 2/40 — Train Loss: 1.2183, Train Acc: 0.5259  |  Val Loss: 1.2044, Val Acc: 0.5297
------------------------------------------------------------


...

Epoch 12/40 — Train Loss: 1.2083, Train Acc: 0.5318  |  Val Loss: 1.2028, Val Acc: 0.5322
------------------------------------------------------------


Epoch 13 [Val]: 100%|██████████| 29/29 [00:03<00:00,  7.66it/s]


Epoch 13/40 — Train Loss: 1.2159, Train Acc: 0.5283  |  Val Loss: 1.2077, Val Acc: 0.5277
------------------------------------------------------------


Epoch 14 [Train]:  77%|███████▋  | 174/225 [01:10<00:20,  2.47it/s]


KeyboardInterrupt: 

#Targeted oversampling of underperforming classes

In [ ]:
import numpy as np
import pandas as pd

def targeted_oversampling(df, target_count, img_shape=(48,48)):
    """
    Upsample each emotion class:
      – if emotion in [0,1,2,6], new_target = original_count * 2
      – otherwise           , new_target = target_count
    Applies a random ±10 intensity shift to each augmented image.
    """
    def augment(pix_str):
        arr   = np.fromstring(pix_str, sep=' ', dtype=np.uint8).reshape(img_shape)
        shift = np.random.randint(-10, 11)
        arr   = np.clip(arr.astype(int) + shift, 0, 255).astype(np.uint8)
        return ' '.join(map(str, arr.ravel()))

    # define which emotion labels to double
    double_classes = {0, 1, 2, 6}

    parts = [df]
    for emo, grp in df.groupby('emotion'):
        n_orig = len(grp)
        # decide how many we want in the end
        if emo in double_classes:
            new_target = n_orig * 2
        else:
            new_target = target_count

        # only augment if we need more
        if new_target > n_orig:
            n_extra = new_target - n_orig
            extra  = grp.sample(n_extra, replace=True).copy()
            extra['pixels'] = extra['pixels'].map(augment)
            parts.append(extra)

    return pd.concat(parts, ignore_index=True)


In [ ]:
# DataLoaders
max_count = df_train['emotion'].value_counts().max()
balanced_train = targeted_oversampling(df_train, target_count=max_count)

#train_ds = FERDataset(df_train)
train_ds = FERDataset(balanced_train)
val_ds = FERDataset(df_val)
test_ds = FERDataset(df_test)

batch_size = 256
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
val_dl = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)
test_dl = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=2)

# Class names
class_names = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

In [ ]:
balanced_train['emotion'].value_counts()

,count
emotion,
6,9930
2,8194
0,7990
4,7215
3,7215
5,7215
1,872


In [ ]:
#improved_model = ImprovedModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(improved_model.parameters(), lr=1e-3)
trained = train_model(
    improved_model,
    train_dl,
    val_dl,
    criterion,
    optimizer,
    device,
    epochs=40,
    class_names=class_names
)

#Targeted oversampling run1: https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/fufcp077?nw=nwuserdimna21
#run2 (continued): https://wandb.ai/dimna21-free-university-of-tbilisi-/ML_Assignment4/runs/gltl4ney
# Tops out at 61%/58%

f1_Angry,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇████
f1_Disgust,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1_Fear,▁▁▃▁▁▁▄▅▃▅▄▅▅▆▆▇█▇▆▅▅▄▄▄▄▄▅▅▆▅▅▆▆▆▆▆▇▆█▆
f1_Happy,▁▁▁▁▁▁▁▆▆▇▇▇▇▇▇█████████████████████████
f1_Neutral,▁▁▁▁▁▂▂▁▃▃▄▄▄▄▄▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇██
f1_Sad,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▂▆▃▅▆▆▅▅▆▆▆▇▆▆▆▆▇▇▇███
f1_Surprise,▂▃▁▄▂▆▆▇▇▇██████████████████████████████
train_acc,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████
train_loss,█▇▇▇▇▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▂▂▄▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██▇█████████
val_loss,█████▇▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁


Epoch 1 [Val]: 100%|██████████| 15/15 [00:00<00:00, 18.58it/s]


Epoch 1/40 — Train Loss: 1.2425, Train Acc: 0.5240  |  Val Loss: 1.2003, Val Acc: 0.5366
------------------------------------------------------------


Epoch 2 [Val]: 100%|██████████| 15/15 [00:01<00:00, 12.74it/s]


Epoch 2/40 — Train Loss: 1.2296, Train Acc: 0.5290  |  Val Loss: 1.2111, Val Acc: 0.5380
------------------------------------------------------------


...

Epoch 38/40 — Train Loss: 1.0810, Train Acc: 0.6052  |  Val Loss: 1.1568, Val Acc: 0.5754
------------------------------------------------------------


Epoch 39 [Val]: 100%|██████████| 15/15 [00:00<00:00, 28.82it/s]


Epoch 39/40 — Train Loss: 1.0805, Train Acc: 0.6047  |  Val Loss: 1.1654, Val Acc: 0.5770
------------------------------------------------------------


Epoch 40 [Val]: 100%|██████████| 15/15 [00:01<00:00, 10.90it/s]


Epoch 40/40 — Train Loss: 1.0709, Train Acc: 0.6073  |  Val Loss: 1.1593, Val Acc: 0.5745
------------------------------------------------------------


In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix

# switch to eval mode
improved_model.eval()

all_preds, all_labels = [], []

with torch.no_grad():
    for X, y in test_dl:
        X, y = X.to(device), y.to(device)
        out = improved_model(X)
        preds = out.argmax(dim=1)
        all_preds .extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

# 1) Full report
print("Test Classification Report")
print(classification_report(all_labels, all_preds, target_names=class_names))

# 2) Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Test Confusion Matrix:\n", cm)

# 3) (Optional) save raw preds
import pandas as pd
test_df = df_test.copy().reset_index(drop=True)
test_df['predicted_emotion'] = [class_names[p] for p in all_preds]
test_df.to_csv("fer2013_test_predictions.csv", index=False)
print("Saved predictions to fer2013_test_predictions.csv")


Test Classification Report
              precision    recall  f1-score   support

       Angry       0.48      0.55      0.51       491
     Disgust       0.00      0.00      0.00        55
        Fear       0.47      0.28      0.35       528
       Happy       0.82      0.83      0.82       879
         Sad       0.47      0.35      0.40       594
    Surprise       0.68      0.84      0.75       416
     Neutral       0.50      0.70      0.58       626

    accuracy                           0.60      3589
   macro avg       0.49      0.51      0.49      3589
weighted avg       0.58      0.60      0.58      3589

Test Confusion Matrix:
 [[270   0  31  27  51  18  94]
 [ 42   0   9   2   0   1   1]
 [ 88   0 146  24  86  86  98]
 [ 19   0  14 730  34  30  52]
 [ 81   0  69  46 205  14 179]
 [  9   0  23  16   3 351  14]
 [ 50   0  16  50  57  15 438]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Saved predictions to fer2013_test_predictions.csv
